In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from scipy.optimize import minimize
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# 1. DATA PREPARATION (Extracted from your files)
# History of inputs for Function 4
X_train = np.array([
    [0.5, 0.5, 0.5, 0.5],                        # Week 1
    [0.3, 0.3, 0.3, 0.3],                        # Week 2
    [0.444081, 0.290589, 0.353209, 1.246981]     # Week 3
])

# History of outputs for Function 4
y_train = np.array([
    -3.985750969810286,
    -4.306383726360995,
    -30.12883896917845
])

# 2. MODEL DEFINITION
# We use a Matern kernel which is robust for non-smooth functions, 
# plus a WhiteKernel to handle noise/variance.
kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)

# 3. TRAINING
gp.fit(X_train, y_train)

# 4. ACQUISITION FUNCTION (Upper Confidence Bound - UCB)
# We want to maximize the function, so we maximize UCB.
# Scipy minimizes, so we return negative UCB.
def acquisition_func(x):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    kappa = 1.96  # Exploration/Exploitation balance
    ucb = mu + kappa * sigma
    return -ucb.item()

# 5. OPTIMIZATION
# Constraints: Inputs must be positive. 
# We bound the search to reasonable limits based on previous data (0.0 to 1.0) 
# to avoid the penalty seen at 1.2.
bounds = [(0.01, 1.0), (0.01, 1.0), (0.01, 1.0), (0.01, 1.0)]

# Run multiple random starts to avoid local minima
best_x = None
best_score = float('inf')

for i in range(50):
    x0 = np.random.uniform(0.2, 0.8, 4) # Start search around the known "good" area
    res = minimize(acquisition_func, x0=x0, bounds=bounds, method='L-BFGS-B')
    
    if res.fun < best_score:
        best_score = res.fun
        best_x = res.x

print("--- Data Summary ---")
print(f"Week 1 Best: {y_train[0]} (Input: 0.5)")
print(f"Week 2:      {y_train[1]} (Input: 0.3)")
print(f"Week 3:      {y_train[2]} (Input: Mixed/High)")
print("\n--- Recommendation for Week 4 ---")
print("Based on Gaussian Process with UCB Acquisition:")
print(f"Predicted Input: {np.array2string(best_x, separator=', ')}")

--- Data Summary ---
Week 1 Best: -3.985750969810286 (Input: 0.5)
Week 2:      -4.306383726360995 (Input: 0.3)
Week 3:      -30.12883896917845 (Input: Mixed/High)

--- Recommendation for Week 4 ---
Based on Gaussian Process with UCB Acquisition:
Predicted Input: [0.50897635, 0.60394654, 0.56520169, 0.01219807]
